# Parsing na Pythonie
___
![](page.png)
![](page1.png)

In [195]:
#biblioteki dla parsingu na Pythonie
from bs4 import BeautifulSoup
import requests
import json
import csv

In [196]:
# link do strony
url = 'https://health-diet.ru/table_calorie/?utm_source=leftMenu&utm_medium=table_calorie'

#(Opcjonalnie) hedearsy aby strona internetowa widziala ze nie jestem botem
headers = {
    "accept": "*/*",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
}

req = requests.get(url,headers = headers)

scr = req.text #kod strony internetowej w formacie html

### Zapisuje strone w formacie html do folderu

In [93]:

with open("main.html", "w") as file:
    file.write(scr)
    
with open("main.html") as file:
    scr = file.read()

In [197]:
#tworze obiekt klasy BeautifulSoup
soup = BeautifulSoup(scr,'lxml')

all_categories_dict = {} 
all_categories = soup.find_all(class_ = "mzr-tc-group-item-href")
for item in all_categories:
        item_text = item.text
        item_hrefs = "https://health-diet.ru" + item.get("href")
        all_categories_dict[item_text] = item_hrefs

In [209]:
all_categories_dict

{'Баранина и дичь': 'https://health-diet.ru/base_of_food/food_24507/',
 'Бобовые': 'https://health-diet.ru/base_of_food/food_24523/',
 'Вода и напитки': 'https://health-diet.ru/base_of_food/food_24509/',
 'Говядина и телятина': 'https://health-diet.ru/base_of_food/food_24502/',
 'Грибы': 'https://health-diet.ru/base_of_food/food_24513/',
 'Детское питание': 'https://health-diet.ru/base_of_food/food_24526/',
 'Жиры и масла': 'https://health-diet.ru/base_of_food/food_24515/',
 'Кондитерские изделия': 'https://health-diet.ru/base_of_food/food_24525/',
 'Крупы, мука, макароны': 'https://health-diet.ru/base_of_food/food_24522/',
 'Молочные продукты': 'https://health-diet.ru/base_of_food/food_24519/',
 'Напитки алкогольные': 'https://health-diet.ru/base_of_food/food_24508/',
 'Овощи и зелень': 'https://health-diet.ru/base_of_food/food_24512/',
 'Орехи и семена': 'https://health-diet.ru/base_of_food/food_24517/',
 'Полуфабрикаты': 'https://health-diet.ru/base_of_meals/meals_21253/',
 'Птица':

### Tworze i odczyuje plik w formacie json, w ktorym znajduje sie slownik ze wszystkimi kategoriami produktow
* `json.dump()` - sluzy do zapisywania pliku w formacie json
* `json.load()` - sluzy do odczyty pliku w formacie json

In [210]:
with open("all_categories_dict.json", "w") as file:
    json.dump(all_categories_dict, file, indent=4, ensure_ascii=False)

with open("all_categories_dict.json") as file:
    all_categories = json.load(file)

### W danym przykladzie nie robie parser na cala strone, a tylko na pierwsze 10 elementow (`if counter < 10`)

In [215]:
count = 0
#zamieniam znaki [" ", ",","'","-"] na jeden znak "_" dla wiekszej czytelnosci
for categories_name, categories_href in all_categories.items():
    if count < 10:
        rep = [" ", ",","'","-"]
        for item in rep:
            if item in categories_name:
                categories_name = categories_name.replace(item,"_")
        
        #dostaje kod dla kadzej strony internetowej
        req = requests.get(url = categories_href,headers = headers)
        src = req.text
        
        #zapisuje kazda strone do osobnego pliku w formacie html
        with open(f"data/{count}_{categories_name}.html", 'w') as file:
            file.write(src)
        #odczytuje plik z kodem strony   
        with open(f"data/{count}_{categories_name}.html") as file:
            src = file.read()
        
        #tworze dla kazdej strony nowy obiekt klasy BeaufifulSoap w jakim znajduje sie kod strony
        soup = BeautifulSoup(src,'lxml')

        #dostaje tytuly tablicy jaka znajduje sie na stronie:
        head_table= soup.find(class_= "mzr-tc-group-table").find("tr").find_all("th")
        for item in head_table:
            product = head_table[0].text
            kkal = head_table[1].text
            proteins = head_table[2].text
            fats = head_table[3].text
            carbohydrates = head_table[4].text
        
        #zapisuje tytuly do pliku csv za pomoca writer.writerow()
        with open(f"data/{count}_{categories_name}.csv",'w', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(
                    (
                        product,
                        kkal,
                        proteins,
                        fats,
                        carbohydrates
                    )
            )
        
        #zapisuje wszystkie dane kazdej karegorii
        all_products = soup.find(class_= "mzr-tc-group-table").find("tbody").find_all("tr")
        products_info = []
        for item in all_products:
            product_tds = item.find_all("td")
            title = product_tds[0].find("a").text
            kkal = product_tds[1].text
            proteins = product_tds[2].text
            fats = product_tds[3].text
            carbohydrates = product_tds[4].text
            
            products_info.append(
                    {
                          "Title": title,
                          "Kkal" : kkal,
                          "Fats" : fats,
                          "Carbohydrates" : carbohydrates
                    }
            )
            
            #zapisuje wszystkie dane do pliku csv 
            with open(f"data/{count}_{categories_name}.csv", "a", encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(
                    (
                        title,
                        kkal,
                        proteins,
                        fats,
                        carbohydrates
                    )
                )
            #tworze dla kazdej kategorii osobny plik formaty json
            with open(f"{count}_{categories_name}.json", "w", encoding='utf-8') as file:
                json.dump(products_info, file, indent= 4, ensure_ascii=False)
    count += 1